In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Initialize DB and Tables

In [18]:
from package.databases.initialize import init_db_and_tables

init_db_and_tables()

In [19]:
from sqlmodel import Session, select, Text
from typing import List
from package.databases.session import get_session, Depends
from package.databases.models.user import User

# CRUD

## Create

In [20]:
def create_user(user:User, session: Session = Depends(get_session)):
    session.add(user)
    session.commit()
    session.refresh(user)
    session.close()
    return user

bank = User(username="bank", password="test", email="bank.bank@bank.com")
bonk = User(username="bonk", password="test", email="bonk.bonk@bank.com")
users = [bank, bonk]
for user in users:
    print(create_user(user=user))

password='test' email='bank.bank@bank.com' created_dt=datetime.datetime(2025, 6, 21, 18, 0, 36, 58347) username='bank' user_id=1 updated_dt=datetime.datetime(2025, 6, 21, 18, 0, 36, 58347)
password='test' email='bonk.bonk@bank.com' created_dt=datetime.datetime(2025, 6, 21, 18, 0, 36, 58347) username='bonk' user_id=2 updated_dt=datetime.datetime(2025, 6, 21, 18, 0, 36, 58347)


In [21]:
def read_user(user_id: int, session=Depends(get_session)):
    statement = select(User).where(User.user_id == user_id)
    results = session.exec(statement).all()
    session.close()
    return results

read_user(1)

[User(password='test', email='bank.bank@bank.com', created_dt=datetime.datetime(2025, 6, 21, 18, 0, 36, 58347), username='bank', user_id=1, updated_dt=datetime.datetime(2025, 6, 21, 18, 0, 36, 58347))]

In [22]:
def read_users(session=Depends(get_session)):
    statement = select(User)
    results = session.exec(statement).all()
    session.close()
    return results

read_users()

[User(password='test', email='bank.bank@bank.com', created_dt=datetime.datetime(2025, 6, 21, 18, 0, 36, 58347), username='bank', user_id=1, updated_dt=datetime.datetime(2025, 6, 21, 18, 0, 36, 58347)),
 User(password='test', email='bonk.bonk@bank.com', created_dt=datetime.datetime(2025, 6, 21, 18, 0, 36, 58347), username='bonk', user_id=2, updated_dt=datetime.datetime(2025, 6, 21, 18, 0, 36, 58347))]

## Update

In [23]:
from datetime import datetime, timezone

def update_user_email(user_id: int, new_email: str, session: Session = Depends(get_session)):
    user = session.get(User, user_id)
    if user:
        user.email = new_email
        user.updated_dt = datetime.now(timezone.utc)  # ✅ update timestamp
        session.add(user)
        session.commit()
        session.refresh(user)
        session.close()
    return user

update_user_email(user_id=1, new_email="bankbank@bank.com")

User(password='test', email='bankbank@bank.com', created_dt=datetime.datetime(2025, 6, 21, 18, 0, 36, 58347), username='bank', user_id=1, updated_dt=datetime.datetime(2025, 6, 21, 18, 0, 37, 474388))

In [24]:
from package.databases.models import Workspace


def create_workspace(workspace: Workspace, session: Session = Depends(get_session)):
    session.add(workspace)
    session.commit()
    session.refresh(workspace)
    session.close()
    return workspace

workspace = Workspace(
    workspace_name="test 1",
    owner_id=1
)

create_workspace(workspace)

Workspace(workspace_name='test 1', owner_id=1, workspace_id=1, updated_dt=datetime.datetime(2025, 6, 21, 18, 0, 37, 672053), created_dt=datetime.datetime(2025, 6, 21, 18, 0, 37, 672053))

In [25]:
workspace = Workspace(
    workspace_name="test 2",
    owner_id=2
)

workspace = create_workspace(workspace)
workspace

Workspace(workspace_name='test 2', owner_id=2, workspace_id=2, updated_dt=datetime.datetime(2025, 6, 21, 18, 0, 37, 939523), created_dt=datetime.datetime(2025, 6, 21, 18, 0, 37, 939523))

In [26]:
def read_workspace(workspace_id:int, session:Session=Depends(get_session)):
    workspace = session.get(Workspace, workspace_id)
    session.close()
    return workspace

def read_workspaces(session:Session=Depends(get_session)):
    statement = select(Workspace)
    results = session.exec(statement).all()
    session.close()
    return results

read_workspace(workspace_id=2)

Workspace(workspace_name='test 2', owner_id=2, workspace_id=2, updated_dt=datetime.datetime(2025, 6, 21, 18, 0, 37, 939523), created_dt=datetime.datetime(2025, 6, 21, 18, 0, 37, 939523))

In [27]:
read_workspaces()

[Workspace(workspace_name='test 1', owner_id=1, workspace_id=1, updated_dt=datetime.datetime(2025, 6, 21, 18, 0, 37, 672053), created_dt=datetime.datetime(2025, 6, 21, 18, 0, 37, 672053)),
 Workspace(workspace_name='test 2', owner_id=2, workspace_id=2, updated_dt=datetime.datetime(2025, 6, 21, 18, 0, 37, 939523), created_dt=datetime.datetime(2025, 6, 21, 18, 0, 37, 939523))]

## Delete

In [28]:
def delete_user(user_id: int, session: Session = Depends(get_session)):
    user = session.get(User, user_id)
    if user:
        session.delete(user)
        session.commit()
        session.close()
        return True
    return False

delete_user(user_id=2)


True

In [29]:
read_users()

[User(password='test', email='bankbank@bank.com', created_dt=datetime.datetime(2025, 6, 21, 18, 0, 36, 58347), username='bank', user_id=1, updated_dt=datetime.datetime(2025, 6, 21, 18, 0, 37, 474388))]

In [30]:
read_workspaces()

[Workspace(workspace_name='test 1', owner_id=1, workspace_id=1, updated_dt=datetime.datetime(2025, 6, 21, 18, 0, 37, 672053), created_dt=datetime.datetime(2025, 6, 21, 18, 0, 37, 672053))]

# Drop the selected table

In [12]:
from package.databases.drop import drop_table

drop_table(User)

✅ Dropped 'user' table.


# Drop all tables

In [16]:
from package.databases.drop import drop_all_tables

drop_all_tables()

✅ All tables dropped.
